# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [35]:
# import libraries
import re
import pandas as pd
import numpy as np
import nltk
import pickle

from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator,TransformerMixin


In [36]:
# download nltk data
nltk.download(['punkt', 'wordnet', 'omw-1.4', 'stopwords'])

# nltk.download()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lajos\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lajos\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\lajos\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lajos\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [37]:
# load data from database
engine = create_engine('sqlite:///../data/disaster_messages.db')
df = pd.read_sql_table('disaster_messages', engine)
X = df['message']
y = df.drop(['message', 'genre', 'id', 'original'], axis=1)

In [38]:
df.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,...,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000
mean,15224.871333,0.766470,0.170666,0.004501,0.414267,0.079496,0.050086,0.027618,0.017967,0.032806,...,0.011787,0.043906,0.278352,0.082205,0.093191,0.010757,0.093649,0.020217,0.052489,0.193591
std,8827.053788,0.423085,0.376224,0.066941,0.492604,0.270517,0.218126,0.163878,0.132833,0.178131,...,0.107929,0.204890,0.448196,0.274682,0.290705,0.103160,0.291345,0.140746,0.223015,0.395120
min,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7446.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15663.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22924.500000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [39]:
y.describe()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.0,...,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000
mean,0.766470,0.170666,0.004501,0.414267,0.079496,0.050086,0.027618,0.017967,0.032806,0.0,...,0.011787,0.043906,0.278352,0.082205,0.093191,0.010757,0.093649,0.020217,0.052489,0.193591
std,0.423085,0.376224,0.066941,0.492604,0.270517,0.218126,0.163878,0.132833,0.178131,0.0,...,0.107929,0.204890,0.448196,0.274682,0.290705,0.103160,0.291345,0.140746,0.223015,0.395120
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [40]:
# X and Y shape
print(f'X shape: {X.shape}')
print(f'y shape: {y.shape}')

# Y column names
print('y column names:', y.columns)

X shape: (26215,)
y shape: (26215, 36)
y column names: Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')


### 2. Write a tokenization function to process your text data

In [41]:
def tokenize(text, should_remove_stopwords=True):
    '''
    Tokenize text.

    Input:
      text: text to tokenize
      should_remove_stopwords: whether to remove stop words (the, an, in etc)

    Output:
      words: list of lemmatized words
    '''
    text = text.lower()
    text = re.sub(r'[^a-z]', " ", text)

    tokens = nltk.word_tokenize(text)

    lemmatizer = nltk.WordNetLemmatizer()
    stop_words = []
    if should_remove_stopwords:
        stop_words = nltk.corpus.stopwords.words("english")

    tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words]

    return tokens

In [42]:
for message in X[:5]:
    tokens = tokenize(message, should_remove_stopwords=False)
    tokens_no_stopwords = tokenize(message, should_remove_stopwords=True)
    print(message)
    print(tokens)
    print(tokens_no_stopwords)
    print('')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'a', 'cold', 'front', 'from', 'cuba', 'that', 'could', 'pas', 'over', 'haiti']
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

Is the Hurricane over or is it not over
['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over']
['hurricane']

Looking for someone but no name
['looking', 'for', 'someone', 'but', 'no', 'name']
['looking', 'someone', 'name']

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', 'destroyed', 'only', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately']
['un', 'report', 'leogane', 'destroyed', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately']

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'of', 'haiti', 'rest', 'of', 'the', 'country', 'today', 'and', 'tonight']
['say', 'west', '

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [69]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier(random_state=42)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000253FA8C3040>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(random_state=42)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [71]:
# test prediction
y_test_pred = pipeline.predict(X_test)

Classification report

In [72]:
print(classification_report(y_test.values, y_test_pred, zero_division=0, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.81      0.97      0.88      3998
               request       0.79      0.50      0.61       891
                 offer       0.00      0.00      0.00        24
           aid_related       0.77      0.60      0.67      2164
          medical_help       0.63      0.26      0.37       435
      medical_products       0.64      0.30      0.41       279
     search_and_rescue       0.61      0.17      0.26       136
              security       0.14      0.02      0.04        96
              military       0.57      0.30      0.40       158
           child_alone       0.00      0.00      0.00         0
                 water       0.75      0.65      0.70       335
                  food       0.81      0.67      0.74       584
               shelter       0.76      0.56      0.65       468
              clothing       0.66      0.33      0.44        70
                 money       0.58      

### 6. Improve your model
Use grid search to find better parameters. 

In [74]:
# show available pipeline parameters
print(pipeline.get_params().keys())

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__algorithm', 'clf__estimator__base_estimator', 'clf__estimator__learning_rate', 'clf__estimator__n_estimators', 'clf__estimator__random_state', 'clf__estimator', 'clf__n_jobs'])


In [75]:
parameters = {'clf__estimator__learning_rate': [0.25, 0.5, 1.0],
              'clf__estimator__n_estimators': [50, 100, 200]}

cv = GridSearchCV(pipeline, param_grid=parameters, scoring='f1_micro', n_jobs=-1)
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000253FA8C3040>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier(random_state=42)))]),
             n_jobs=-1,
             param_grid={'clf__estimator__learning_rate': [0.25, 0.5, 1.0],
                         'clf__estimator__n_estimators': [50, 100, 200]},
             scoring='f1_micro')

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [76]:
# test prediction
y_test_cv_pred = cv.predict(X_test)

In [77]:
print(classification_report(y_test.values, y_test_cv_pred, zero_division=0, target_names=y.columns.values))
print(f'best parameters: {cv.best_params_}')

                        precision    recall  f1-score   support

               related       0.82      0.96      0.89      3998
               request       0.82      0.50      0.62       891
                 offer       0.00      0.00      0.00        24
           aid_related       0.80      0.60      0.69      2164
          medical_help       0.64      0.22      0.33       435
      medical_products       0.68      0.26      0.38       279
     search_and_rescue       0.74      0.12      0.21       136
              security       0.11      0.01      0.02        96
              military       0.56      0.26      0.35       158
           child_alone       0.00      0.00      0.00         0
                 water       0.75      0.65      0.69       335
                  food       0.82      0.74      0.78       584
               shelter       0.78      0.55      0.65       468
              clothing       0.71      0.34      0.46        70
                 money       0.58      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [78]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    '''
    A custom transformer class that tests if the
    the sentence starts with a verb.
    '''
    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            if not len(pos_tags):
                return False
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [79]:
pipeline_improved = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
            ('stverb', StartingVerbExtractor())
        ])),
        ('clf', MultiOutputClassifier(AdaBoostClassifier(random_state=42)))
    ])


In [80]:
pipeline_improved.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x00000253FA8C3040>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())])),
                                                ('stverb',
                                                 StartingVerbExtractor())])),
                ('clf',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(random_state=42)))])

In [81]:
# test prediction
y_test_imp_pred = pipeline_improved.predict(X_test)

In [82]:
print(classification_report(y_test.values, y_test_imp_pred, zero_division=0, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.81      0.97      0.88      3998
               request       0.79      0.50      0.61       891
                 offer       0.00      0.00      0.00        24
           aid_related       0.77      0.60      0.67      2164
          medical_help       0.63      0.26      0.37       435
      medical_products       0.64      0.30      0.41       279
     search_and_rescue       0.61      0.17      0.26       136
              security       0.14      0.02      0.04        96
              military       0.57      0.30      0.40       158
           child_alone       0.00      0.00      0.00         0
                 water       0.77      0.67      0.71       335
                  food       0.81      0.67      0.74       584
               shelter       0.76      0.56      0.65       468
              clothing       0.66      0.33      0.44        70
                 money       0.58      

### 9. Export your model as a pickle file

In [83]:
with open('../models/classifier.pkl', 'wb') as pickle_file:
    pickle.dump(pipeline_improved, pickle_file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.